# Infer-17-Decision-Networks : Reseaux de Decision

**Serie** : Programmation Probabiliste avec Infer.NET (17/20)  
**Duree estimee** : 55 minutes  
**Prerequis** : Notebooks 3 (Factor Graphs), 14-16 (Utilite)

---

## Objectifs

- Etendre les reseaux bayesiens avec **noeuds de decision et d'utilite**
- Calculer la **politique optimale** dans un reseau de decision
- Comprendre les **arcs informationnels**
- Modeliser des **decisions sequentielles**

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-16-Decision-Multi-Attribute](Infer-16-Decision-Multi-Attribute.ipynb) | [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb) |

---

## 1. Des Reseaux Bayesiens aux Reseaux de Decision

### Rappel : Reseaux Bayesiens

Un reseau bayesien represente des **relations probabilistes** entre variables :

```
  [Maladie] ----> [Symptome]
       \--------> [Test]
```

### Extension : Reseaux de Decision (Influence Diagrams)

Un reseau de decision ajoute :

- **Noeuds de decision** : choix de l'agent
- **Noeuds d'utilite** : consequences des choix
- **Arcs informationnels** : ce que l'agent sait au moment de decider

```
  [Maladie] ----> [Test Result]
       |              |
       v              v
  <Traitement> ----> ((Utilite))
```

### Configuration de l'environnement

Avant de modeliser des reseaux de decision, nous chargeons la bibliotheque Infer.NET. Cette cellule est necessaire pour toutes les demonstrations de ce notebook.

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge !


## 2. Types de Noeuds

Les reseaux de decision utilisent une notation graphique standardisee (Howard, 1984) qui distingue clairement les differents types de noeuds. Cette convention visuelle permet de lire immediatement la structure du probleme decisionnel.

### Convention graphique

| Type | Forme | Symbole | Description | Exemple |
|------|-------|---------|-------------|---------|
| **Chance** | Ovale/Cercle | [X] | Variable aleatoire non controlee | Maladie, Marche, Meteo |
| **Decision** | Rectangle | <D> | Choix controle par l'agent | Traiter, Investir, Acheter |
| **Utilite** | Losange/Hexagone | ((U)) | Fonction de valeur/payoff | Profit, Sante, Satisfaction |

### Arcs et leur signification

Chaque type d'arc encode une relation specifique dans le probleme :

| Type d'arc | Signification | Interpretation |
|------------|---------------|----------------|
| Chance → Chance | Dependance probabiliste | P(B\|A) - A influence B |
| Chance → Decision | **Arc informationnel** | L'agent observe A avant de choisir D |
| Chance → Utilite | La variable affecte l'utilite | U depend de la valeur de A |
| Decision → Chance | La decision influence le resultat | L'action D modifie P(B) |
| Decision → Utilite | Cout/benefice direct de la decision | U inclut le cout de D |

### Ordre temporel implicite

Dans un reseau de decision bien forme :
1. Les noeuds de decision sont **totalement ordonnes** (D1 avant D2 avant D3...)
2. Un arc informationnel vers Di signifie que la variable est connue **avant** Di
3. Pas de cycles impliquant des noeuds de decision

La cellule suivante definit une structure de donnees simple pour representer les noeuds d'un reseau de decision et illustre la notation graphique sur un exemple medical.

In [2]:
// Representation d'un reseau de decision simple

public enum NodeType { Chance, Decision, Utility }

public class DecisionNode
{
    public string Name { get; set; }
    public NodeType Type { get; set; }
    public List<string> Parents { get; set; } = new List<string>();
    
    public string ToSymbol()
    {
        return Type switch
        {
            NodeType.Chance => $"[{Name}]",
            NodeType.Decision => $"<{Name}>",
            NodeType.Utility => $"(({Name}))",
            _ => Name
        };
    }
}

// Exemple : Decision medicale
var network = new List<DecisionNode>
{
    new DecisionNode { Name = "Maladie", Type = NodeType.Chance },
    new DecisionNode { Name = "Test", Type = NodeType.Chance, Parents = { "Maladie" } },
    new DecisionNode { Name = "Traiter", Type = NodeType.Decision, Parents = { "Test" } },
    new DecisionNode { Name = "Sante", Type = NodeType.Chance, Parents = { "Maladie", "Traiter" } },
    new DecisionNode { Name = "Utilite", Type = NodeType.Utility, Parents = { "Sante", "Traiter" } },
};

Console.WriteLine("Reseau de decision : Diagnostic Medical\n");
foreach (var node in network)
{
    string parents = node.Parents.Any() ? $" <- {string.Join(", ", node.Parents)}" : "";
    Console.WriteLine($"  {node.ToSymbol()}{parents}");
}

Reseau de decision : Diagnostic Medical

  [Maladie]
  [Test] <- Maladie
  <Traiter> <- Test
  [Sante] <- Maladie, Traiter
  ((Utilite)) <- Sante, Traiter


### Interpretation de la structure

Le reseau ci-dessus represente un probleme de **diagnostic medical** classique :

| Noeud | Type | Role |
|-------|------|------|
| `[Maladie]` | Chance | Etat du patient (inconnu) |
| `[Test]` | Chance | Resultat observable, depend de Maladie |
| `<Traiter>` | Decision | Choix du medecin, base sur le resultat du test |
| `[Sante]` | Chance | Etat futur, depend de Maladie ET de la decision |
| `((Utilite))` | Utilite | Valeur finale, depend de Sante et du cout du traitement |

> **Point cle** : L'arc `Test -> Traiter` est un **arc informationnel**. Il indique que le medecin connait le resultat du test avant de decider. Sans cet arc, le medecin devrait decider "en aveugle".

La cellule suivante illustre concretement l'impact des arcs informationnels en comparant deux scenarios : decision sans information (a priori) et decision avec information (apres observation du test).

## 3. Arcs Informationnels

Les arcs informationnels sont fondamentaux pour modeliser **ce que l'agent sait** au moment de prendre une decision. Ils capturent l'asymetrie d'information qui rend les problemes de decision interessants.

### Concept cle

Un arc informationnel vers un noeud de decision indique **ce que l'agent sait** au moment de prendre cette decision. C'est la difference entre agir en aveugle et agir en connaissance de cause.

### Exemple

```
[Resultat Test] ----> <Traitement>
```

Signifie : L'agent connait le resultat du test **avant** de decider du traitement.

**Absence d'arc** = L'agent ne connait **pas** la valeur de la variable.

### Impact sur la politique

| Situation | Politique | Complexite |
|-----------|-----------|------------|
| Aucune information | Action unique | 1 decision |
| Test observe | Action conditionnelle au test | 2^k decisions (k = bits d'information) |
| Information parfaite | Action conditionnelle a l'etat | Action optimale par etat |

### No-forgetting assumption

Dans les decisions sequentielles, on suppose generalement que l'agent **n'oublie pas** :
- Toutes les observations passees restent accessibles
- Toutes les decisions passees sont connues

Cette hypothese simplifie l'analyse mais n'est pas toujours realiste (memoire limitee, cout de stockage).

### Valeur de l'information

L'ajout d'un arc informationnel peut **augmenter** l'utilite esperee. La difference est la **valeur de l'information** que nous etudierons dans le notebook suivant (Infer-18).

In [3]:
// Impact des arcs informationnels

Console.WriteLine("Scenarios d'information :\n");

// Prior sur la maladie
double pMaladie = 0.1;
double sensibilite = 0.95; // P(test+|malade)
double specificite = 0.90; // P(test-|sain)

// Utilites
double U_traiter_malade = 90;    // Guerison
double U_traiter_sain = 70;      // Effets secondaires inutiles
double U_pasTraiter_malade = 10; // Maladie non traitee
double U_pasTraiter_sain = 100;  // Parfait

// Scenario 1 : Pas d'information (decision a priori)
double EU_traiter_apriori = pMaladie * U_traiter_malade + (1 - pMaladie) * U_traiter_sain;
double EU_pasTraiter_apriori = pMaladie * U_pasTraiter_malade + (1 - pMaladie) * U_pasTraiter_sain;

Console.WriteLine("Scenario 1 : Decision SANS information (pas de test)");
Console.WriteLine($"  P(malade) = {pMaladie:P0}");
Console.WriteLine($"  E[U(traiter)] = {EU_traiter_apriori:F1}");
Console.WriteLine($"  E[U(pas traiter)] = {EU_pasTraiter_apriori:F1}");
Console.WriteLine($"  => Decision : {(EU_traiter_apriori > EU_pasTraiter_apriori ? "TRAITER" : "PAS TRAITER")}\n");

// Scenario 2 : Decision AVEC information (apres test)
// Calculer P(malade|test+) et P(malade|test-) par Bayes
double pTestPositif = pMaladie * sensibilite + (1 - pMaladie) * (1 - specificite);
double pMaladeSachantTestPositif = (pMaladie * sensibilite) / pTestPositif;
double pMaladeSachantTestNegatif = (pMaladie * (1 - sensibilite)) / (1 - pTestPositif);

Console.WriteLine("Scenario 2 : Decision AVEC information (apres test)");
Console.WriteLine($"  P(test+) = {pTestPositif:P1}");
Console.WriteLine($"  P(malade|test+) = {pMaladeSachantTestPositif:P1}");
Console.WriteLine($"  P(malade|test-) = {pMaladeSachantTestNegatif:P1}");

// Decision si test positif
double EU_traiter_testPos = pMaladeSachantTestPositif * U_traiter_malade + (1 - pMaladeSachantTestPositif) * U_traiter_sain;
double EU_pasTraiter_testPos = pMaladeSachantTestPositif * U_pasTraiter_malade + (1 - pMaladeSachantTestPositif) * U_pasTraiter_sain;

Console.WriteLine($"  Si test+ : traiter={EU_traiter_testPos:F1}, pas traiter={EU_pasTraiter_testPos:F1} => {(EU_traiter_testPos > EU_pasTraiter_testPos ? "TRAITER" : "PAS TRAITER")}");

// Decision si test negatif
double EU_traiter_testNeg = pMaladeSachantTestNegatif * U_traiter_malade + (1 - pMaladeSachantTestNegatif) * U_traiter_sain;
double EU_pasTraiter_testNeg = pMaladeSachantTestNegatif * U_pasTraiter_malade + (1 - pMaladeSachantTestNegatif) * U_pasTraiter_sain;

Console.WriteLine($"  Si test- : traiter={EU_traiter_testNeg:F1}, pas traiter={EU_pasTraiter_testNeg:F1} => {(EU_traiter_testNeg > EU_pasTraiter_testNeg ? "TRAITER" : "PAS TRAITER")}");

Scenarios d'information :

Scenario 1 : Decision SANS information (pas de test)
  P(malade) = 10 %
  E[U(traiter)] = 72,0
  E[U(pas traiter)] = 91,0
  => Decision : PAS TRAITER

Scenario 2 : Decision AVEC information (apres test)
  P(test+) = 18,5 %
  P(malade|test+) = 51,4 %
  P(malade|test-) = 0,6 %
  Si test+ : traiter=80,3, pas traiter=53,8 => TRAITER
  Si test- : traiter=70,1, pas traiter=99,4 => PAS TRAITER


La cellule suivante implemente une classe `SimpleDecisionNetwork` qui resout le probleme de diagnostic medical par backward induction. Elle calcule la politique optimale (quelle action prendre pour chaque observation) et l'utilite esperee globale.

### Analyse des resultats : Impact de l'information

Les resultats ci-dessus illustrent le **theoreme fondamental de la valeur de l'information** :

| Scenario | Decision | E[U] | Commentaire |
|----------|----------|------|-------------|
| **Sans test** | Pas traiter | 91.0 | Prior faible (10%), risque acceptable |
| **Avec test+** | Traiter | 80.3 | Posterior eleve (51.4%), traitement justifie |
| **Avec test-** | Pas traiter | 99.4 | Posterior tres faible (0.6%), rassurant |

#### Pourquoi la decision change-t-elle ?

1. **A priori** : P(malade) = 10% est faible. Le cout des effets secondaires (U=70 vs 100) domine.

2. **Test positif** : Le theoreme de Bayes "concentre" la probabilite :
   $$P(\text{malade}|\text{test}+) = \frac{0.10 \times 0.95}{0.185} = 51.4\%$$
   Le risque devient majoritaire, justifiant le traitement.

3. **Test negatif** : La probabilite residuelle (0.6%) est negligeable.

> **Lecon** : L'information ne change pas toujours la decision (si le prior etait de 50%, on traiterait dans tous les cas). La valeur de l'information depend de la **distance entre le prior et les seuils de decision**.

## 4. Calcul de la Politique Optimale

### Algorithme Backward Induction

L'algorithme standard pour resoudre un reseau de decision est la **backward induction** (ou programmation dynamique). L'idee est de resoudre le probleme de la fin vers le debut.

### Algorithme pas a pas

**Etape 0 : Preparation**
- Ordonner les noeuds de decision temporellement : D1 → D2 → ... → Dn
- Identifier les parents informationnels de chaque decision

**Etape 1 : Derniere decision (Dn)**
- Pour chaque configuration possible des parents informationnels de Dn :
  - Calculer E[U | parents, action] pour chaque action possible
  - Selectionner l'action qui maximise E[U]
  - Stocker cette action optimale et sa valeur

**Etape 2 : Remonter (Dn-1, Dn-2, ...)**
- Pour chaque decision precedente :
  - Utiliser les valeurs optimales des decisions suivantes
  - Repeter le processus de maximisation

**Etape 3 : Premiere decision (D1)**
- La derniere iteration donne la decision optimale initiale
- La politique complete est l'ensemble des decisions conditionnelles

### Complexite

| Nombre de decisions | Complexite | Commentaire |
|---------------------|------------|-------------|
| 1 | O(|A| × |S|) | A = actions, S = etats |
| n sequentielles | O(|A|^n × |S|^n) | Exponentielle en n |
| Avec info parfaite | O(|S| × |A|) | Lineaire car decision par etat |

### Pourquoi "backward" ?

On commence par la fin car l'utilite finale depend des decisions futures. En resolvant d'abord les decisions futures, on connait leur contribution a l'utilite esperee, ce qui permet de resoudre les decisions anterieures.

La cellule suivante applique les memes principes a un probleme d'investissement. L'investisseur peut commander une etude de marche (couteuse) avant de decider s'il investit. Le code compare les deux strategies : decision directe vs decision informee.

In [4]:
// Implementation d'un solveur de reseau de decision simple

public class SimpleDecisionNetwork
{
    // Noeud chance : Maladie (M)
    public double P_M { get; set; } = 0.1; // P(malade)
    
    // Noeud chance : Test (T) | M
    public double P_TposGivenM { get; set; } = 0.95;  // Sensibilite
    public double P_TposGivenNotM { get; set; } = 0.10; // 1-Specificite
    
    // Noeud decision : Traiter (D) avec parent informationnel T
    // Options : D=true (traiter), D=false (pas traiter)
    
    // Noeud utilite : U(M, D)
    public double U_traiter_malade { get; set; } = 90;
    public double U_traiter_sain { get; set; } = 70;
    public double U_pasTraiter_malade { get; set; } = 10;
    public double U_pasTraiter_sain { get; set; } = 100;
    
    public double GetUtility(bool malade, bool traiter)
    {
        if (traiter && malade) return U_traiter_malade;
        if (traiter && !malade) return U_traiter_sain;
        if (!traiter && malade) return U_pasTraiter_malade;
        return U_pasTraiter_sain;
    }
    
    public (Dictionary<bool, bool> policy, double expectedUtility) Solve()
    {
        // Politique : Test result -> Decision
        var policy = new Dictionary<bool, bool>();
        
        // P(T+)
        double pTpos = P_M * P_TposGivenM + (1 - P_M) * P_TposGivenNotM;
        double pTneg = 1 - pTpos;
        
        // Pour T+ : calculer decision optimale
        double pM_givenTpos = (P_M * P_TposGivenM) / pTpos;
        double EU_traiter_Tpos = pM_givenTpos * U_traiter_malade + (1 - pM_givenTpos) * U_traiter_sain;
        double EU_pasTraiter_Tpos = pM_givenTpos * U_pasTraiter_malade + (1 - pM_givenTpos) * U_pasTraiter_sain;
        policy[true] = EU_traiter_Tpos > EU_pasTraiter_Tpos;
        double bestU_Tpos = Math.Max(EU_traiter_Tpos, EU_pasTraiter_Tpos);
        
        // Pour T- : calculer decision optimale
        double pM_givenTneg = (P_M * (1 - P_TposGivenM)) / pTneg;
        double EU_traiter_Tneg = pM_givenTneg * U_traiter_malade + (1 - pM_givenTneg) * U_traiter_sain;
        double EU_pasTraiter_Tneg = pM_givenTneg * U_pasTraiter_malade + (1 - pM_givenTneg) * U_pasTraiter_sain;
        policy[false] = EU_traiter_Tneg > EU_pasTraiter_Tneg;
        double bestU_Tneg = Math.Max(EU_traiter_Tneg, EU_pasTraiter_Tneg);
        
        // Utilite esperee totale
        double totalEU = pTpos * bestU_Tpos + pTneg * bestU_Tneg;
        
        return (policy, totalEU);
    }
}

var dn = new SimpleDecisionNetwork();
var (policy, eu) = dn.Solve();

Console.WriteLine("=== Resolution du Reseau de Decision ===");
Console.WriteLine();
Console.WriteLine("Politique optimale :");
Console.WriteLine($"  Si test POSITIF : {(policy[true] ? "TRAITER" : "PAS TRAITER")}");
Console.WriteLine($"  Si test NEGATIF : {(policy[false] ? "TRAITER" : "PAS TRAITER")}");
Console.WriteLine();
Console.WriteLine($"Utilite esperee : {eu:F2}");

=== Resolution du Reseau de Decision ===

Politique optimale :
  Si test POSITIF : TRAITER
  Si test NEGATIF : PAS TRAITER

Utilite esperee : 95,90


### Interpretation de la politique optimale

La classe `SimpleDecisionNetwork` implemente l'algorithme de backward induction de maniere explicite. Le resultat est une **politique conditionnelle** :

| Observation | P(malade\|obs) | Action optimale | E[U\|obs, action*] |
|-------------|----------------|-----------------|-------------------|
| Test positif | 51.4% | TRAITER | 80.3 |
| Test negatif | 0.6% | PAS TRAITER | 99.4 |

L'**utilite esperee globale** de 95.90 est calculee comme :

$$E[U] = P(\text{test}+) \times E[U|\text{test}+, \text{action}^*] + P(\text{test}-) \times E[U|\text{test}-, \text{action}^*]$$

$$E[U] = 0.185 \times 80.3 + 0.815 \times 99.4 = 95.90$$

> **Remarque technique** : Cette utilite (95.90) est superieure a celle sans test (91.0). La difference (4.90) represente la **valeur de l'information** apportee par le test - un concept que nous approfondirons dans le notebook suivant.

## 5. Exemple : Investissement avec Test de Marche

### Scenario

Un investisseur considere un projet. Le marche peut etre favorable ou non.
Il peut commander une etude de marche avant de decider.

```
[Marche] -----> [Etude] -----> <Investir>
    |                              |
    +----------------------------> ((Profit))
```

La cellule suivante implemente une decision sequentielle a deux etapes : d'abord decider si on fait le test, puis decider si on traite. L'algorithme de backward induction resout d'abord la deuxieme decision, puis remonte a la premiere.

In [5]:
// Reseau de decision : Investissement

// Marche (M) : Favorable (F) ou Defavorable (D)
double pFavorable = 0.4;

// Etude de marche (E) : Positive ou Negative
// Qualite de l'etude
double pEposGivenF = 0.8;  // Detecte correctement marche favorable
double pEposGivenD = 0.3;  // Faux positif

// Profits selon marche et decision
double profit_investir_favorable = 500000;    // Gros profit
double profit_investir_defavorable = -200000; // Grosse perte
double profit_pasInvestir = 0;                // Statu quo
double cout_etude = 20000;                    // Cout de l'etude

// Calculer la politique optimale
Console.WriteLine("=== Decision d'Investissement ===");
Console.WriteLine($"P(marche favorable) = {pFavorable:P0}");
Console.WriteLine($"Cout de l'etude = {cout_etude:N0} EUR\n");

// Option 1 : Pas d'etude, decision directe
double EU_investir_direct = pFavorable * profit_investir_favorable + 
                            (1 - pFavorable) * profit_investir_defavorable;
double EU_pasInvestir_direct = profit_pasInvestir;
double bestEU_sansEtude = Math.Max(EU_investir_direct, EU_pasInvestir_direct);

Console.WriteLine("Option 1 : Decision directe (sans etude)");
Console.WriteLine($"  E[Profit(investir)] = {EU_investir_direct:N0} EUR");
Console.WriteLine($"  E[Profit(pas investir)] = {EU_pasInvestir_direct:N0} EUR");
Console.WriteLine($"  => Decision : {(EU_investir_direct > EU_pasInvestir_direct ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  => E[Profit] = {bestEU_sansEtude:N0} EUR\n");

// Option 2 : Avec etude
double pEpos = pFavorable * pEposGivenF + (1 - pFavorable) * pEposGivenD;
double pEneg = 1 - pEpos;

// Si etude positive
double pF_givenEpos = (pFavorable * pEposGivenF) / pEpos;
double EU_investir_Epos = pF_givenEpos * profit_investir_favorable + 
                          (1 - pF_givenEpos) * profit_investir_defavorable;
bool investir_siEpos = EU_investir_Epos > profit_pasInvestir;
double bestEU_Epos = Math.Max(EU_investir_Epos, profit_pasInvestir);

// Si etude negative
double pF_givenEneg = (pFavorable * (1 - pEposGivenF)) / pEneg;
double EU_investir_Eneg = pF_givenEneg * profit_investir_favorable + 
                          (1 - pF_givenEneg) * profit_investir_defavorable;
bool investir_siEneg = EU_investir_Eneg > profit_pasInvestir;
double bestEU_Eneg = Math.Max(EU_investir_Eneg, profit_pasInvestir);

double EU_avecEtude = pEpos * bestEU_Epos + pEneg * bestEU_Eneg - cout_etude;

Console.WriteLine("Option 2 : Avec etude de marche");
Console.WriteLine($"  P(etude positive) = {pEpos:P1}");
Console.WriteLine($"  Si E+ : P(favorable|E+) = {pF_givenEpos:P1} => {(investir_siEpos ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  Si E- : P(favorable|E-) = {pF_givenEneg:P1} => {(investir_siEneg ? "INVESTIR" : "NE PAS INVESTIR")}");
Console.WriteLine($"  => E[Profit avec etude] = {EU_avecEtude:N0} EUR\n");

Console.WriteLine("=== Comparaison ===");
Console.WriteLine($"Sans etude : {bestEU_sansEtude:N0} EUR");
Console.WriteLine($"Avec etude : {EU_avecEtude:N0} EUR");
Console.WriteLine($"=> Valeur de l'etude : {EU_avecEtude - bestEU_sansEtude:N0} EUR");
Console.WriteLine($"=> Decision : {(EU_avecEtude > bestEU_sansEtude ? "COMMANDER L'ETUDE" : "DECIDER DIRECTEMENT")}");

=== Decision d'Investissement ===
P(marche favorable) = 40 %
Cout de l'etude = 20 000 EUR

Option 1 : Decision directe (sans etude)
  E[Profit(investir)] = 80 000 EUR
  E[Profit(pas investir)] = 0 EUR
  => Decision : INVESTIR
  => E[Profit] = 80 000 EUR

Option 2 : Avec etude de marche
  P(etude positive) = 50,0 %
  Si E+ : P(favorable|E+) = 64,0 % => INVESTIR
  Si E- : P(favorable|E-) = 16,0 % => NE PAS INVESTIR
  => E[Profit avec etude] = 104 000 EUR

=== Comparaison ===
Sans etude : 80 000 EUR
Avec etude : 104 000 EUR
=> Valeur de l'etude : 24 000 EUR
=> Decision : COMMANDER L'ETUDE


### Analyse de la decision d'investissement

Les resultats revelent plusieurs phenomenes interessants :

#### Comparaison des strategies

| Strategie | Decision | E[Profit] | Commentaire |
|-----------|----------|-----------|-------------|
| **Sans etude** | Investir directement | 80 000 EUR | Prior P(favorable)=40% suffit |
| **Avec etude** | Conditionnel au resultat | 104 000 EUR | Evite les grosses pertes |

#### Decomposition de la valeur de l'etude

La valeur de l'etude (24 000 EUR) se decompose ainsi :

1. **Gain brut d'information** : L'etude permet d'**eviter d'investir** quand le signal est negatif
   - Sans etude : on investit toujours (EU = 80 000 EUR)
   - Avec E- (50% des cas) : on n'investit pas, evitant une perte esperee

2. **Cout de l'etude** : -20 000 EUR

3. **Valeur nette** : 24 000 EUR

> **Point cle** : L'etude est rentable car elle permet de **changer de decision** dans certains cas. Si P(favorable) etait de 80%, on investirait dans tous les cas (E+ ou E-), et l'etude n'aurait aucune valeur decisionnelle.

#### Conditions pour que l'information ait de la valeur

L'information a de la valeur si et seulement si elle peut **changer la decision optimale**. Dans cet exemple :
- E+ confirme l'investissement (64% favorable)
- E- dissuade l'investissement (16% favorable)

La **qualite** de l'etude (sensibilite/specificite) determine a quel point les posteriors divergent du prior.

L'exemple precedent illustrait une situation avec **une seule decision** apres observation. Dans de nombreux problemes reels, nous devons prendre **plusieurs decisions dans le temps**, chacune pouvant reveler de l'information utile pour les suivantes.

## 6. Decisions Sequentielles

### Ordre des decisions

Quand il y a plusieurs noeuds de decision, leur **ordre temporel** doit etre specifie.

### Exemple : Test puis Traitement

```
<Faire Test?> ----> [Resultat] ----> <Traiter?> ----> ((Utilite))
                                          ^
                                          |
[Maladie] ---------------------------------+
```

Deux decisions sequentielles :
1. D'abord : Faire le test ou non ?
2. Ensuite : Traiter ou non ? (en connaissant le resultat si test fait)

In [6]:
// Decision sequentielle : Faire test puis traiter

// Parametres
double pMaladie_seq = 0.2;
double sensibilite_seq = 0.90;
double specificite_seq = 0.85;
double cout_test = 50;

// Utilites (sans cout test)
double U_traiterMalade = 100;
double U_traiterSain = 60;
double U_pasTraiterMalade = 20;
double U_pasTraiterSain = 100;

// Resolution par backward induction
Console.WriteLine("=== Decision Sequentielle : Test puis Traitement ===\n");

// D'abord : calculer politique optimale pour D2 (traiter) sachant test fait
double pTpos_seq = pMaladie_seq * sensibilite_seq + (1 - pMaladie_seq) * (1 - specificite_seq);

double pM_Tpos = (pMaladie_seq * sensibilite_seq) / pTpos_seq;
double EU_traiter_Tpos = pM_Tpos * U_traiterMalade + (1 - pM_Tpos) * U_traiterSain;
double EU_pasTraiter_Tpos = pM_Tpos * U_pasTraiterMalade + (1 - pM_Tpos) * U_pasTraiterSain;
double bestU_Tpos = Math.Max(EU_traiter_Tpos, EU_pasTraiter_Tpos) - cout_test;

double pM_Tneg = (pMaladie_seq * (1 - sensibilite_seq)) / (1 - pTpos_seq);
double EU_traiter_Tneg = pM_Tneg * U_traiterMalade + (1 - pM_Tneg) * U_traiterSain;
double EU_pasTraiter_Tneg = pM_Tneg * U_pasTraiterMalade + (1 - pM_Tneg) * U_pasTraiterSain;
double bestU_Tneg = Math.Max(EU_traiter_Tneg, EU_pasTraiter_Tneg) - cout_test;

double EU_faireTest = pTpos_seq * bestU_Tpos + (1 - pTpos_seq) * bestU_Tneg;

// Si pas de test : decision a priori
double EU_traiter_apriori = pMaladie_seq * U_traiterMalade + (1 - pMaladie_seq) * U_traiterSain;
double EU_pasTraiter_apriori = pMaladie_seq * U_pasTraiterMalade + (1 - pMaladie_seq) * U_pasTraiterSain;
double EU_pasTest = Math.Max(EU_traiter_apriori, EU_pasTraiter_apriori);

Console.WriteLine("Etape 2 : Politique de traitement si test fait");
Console.WriteLine($"  Si T+ : P(M|T+)={pM_Tpos:P1} => {(EU_traiter_Tpos > EU_pasTraiter_Tpos ? "Traiter" : "Pas traiter")}");
Console.WriteLine($"  Si T- : P(M|T-)={pM_Tneg:P1} => {(EU_traiter_Tneg > EU_pasTraiter_Tneg ? "Traiter" : "Pas traiter")}\n");

Console.WriteLine("Etape 1 : Decision de faire le test");
Console.WriteLine($"  E[U | faire test] = {EU_faireTest:F1}");
Console.WriteLine($"  E[U | pas test] = {EU_pasTest:F1} ({(EU_traiter_apriori > EU_pasTraiter_apriori ? "traiter" : "pas traiter")})\n");

Console.WriteLine("=== Politique Optimale Globale ===");
Console.WriteLine($"D1 : {(EU_faireTest > EU_pasTest ? "FAIRE LE TEST" : "NE PAS FAIRE LE TEST")}");
if (EU_faireTest > EU_pasTest)
{
    Console.WriteLine($"D2 : Si T+ => {(EU_traiter_Tpos > EU_pasTraiter_Tpos ? "TRAITER" : "PAS TRAITER")}");
    Console.WriteLine($"     Si T- => {(EU_traiter_Tneg > EU_pasTraiter_Tneg ? "TRAITER" : "PAS TRAITER")}");
}
else
{
    Console.WriteLine($"D2 : {(EU_traiter_apriori > EU_pasTraiter_apriori ? "TRAITER" : "PAS TRAITER")} (sans test)");
}

=== Decision Sequentielle : Test puis Traitement ===

Etape 2 : Politique de traitement si test fait
  Si T+ : P(M|T+)=60,0 % => Traiter
  Si T- : P(M|T-)=2,9 % => Pas traiter

Etape 1 : Decision de faire le test
  E[U | faire test] = 43,6
  E[U | pas test] = 84,0 (pas traiter)

=== Politique Optimale Globale ===
D1 : NE PAS FAIRE LE TEST
D2 : PAS TRAITER (sans test)


### Analyse de la decision sequentielle

Ce resultat est **contre-intuitif** : malgre un test informatif (sensibilite 90%, specificite 85%), la politique optimale est de **ne pas faire le test**.

#### Pourquoi ne pas faire le test ?

| Facteur | Valeur | Impact |
|---------|--------|--------|
| Prior P(maladie) | 20% | Relativement faible |
| Cout du test | 50 points | Penalise chaque branche |
| Utilite "pas traiter sain" | 100 | Tres attractive |
| Decision a priori optimale | Pas traiter | EU = 84.0 |
| Decision avec test | Conditionnelle | EU = 43.6 (apres cout) |

Le cout du test (50 points) est applique dans **les deux branches** (T+ et T-), ce qui reduit significativement l'utilite esperee.

#### Decomposition du calcul

$$E[U|\text{faire test}] = P(T+) \times (E[U^*|T+] - 50) + P(T-) \times (E[U^*|T-] - 50)$$

Le cout fixe de 50 points fait basculer la decision vers "ne pas tester".

> **Lecon importante** : Dans les decisions sequentielles, le **timing** des couts est crucial. Un cout fixe en amont peut rendre toute la branche sous-optimale, meme si l'information obtenue est de haute qualite.

#### Quand le test deviendrait-il rentable ?

Le test serait rentable si :
- Son cout etait inferieur a environ 40 points, OU
- Le prior P(maladie) etait plus eleve (ex: 40%), OU  
- L'ecart d'utilite "traiter malade vs pas traiter malade" etait plus grand

### Approche pratique avec Infer.NET

Dans la cellule suivante, nous allons implementer un reseau de decision complet en utilisant Infer.NET pour l'inference bayesienne :

1. **Modelisation** : Variables `maladie` et `testPositif` avec leurs dependances
2. **Enumeration** : Pour chaque valeur possible du test (positif/negatif)
3. **Inference** : Calcul de P(maladie|test) via `engine.Infer<Bernoulli>()`
4. **Decision** : Selection de l'action maximisant E[U]

> **Point technique** : Infer.NET n'a pas de noeuds de decision natifs. Nous simulons la resolution en enumerant manuellement les observations possibles et en calculant l'utilite esperee pour chaque action.

## 7. Implementation avec Infer.NET

Infer.NET n'a pas de support natif pour les noeuds de decision, mais nous pouvons :

1. Modeliser les variables de chance
2. Enumerer les decisions
3. Calculer E[U] pour chaque configuration

In [7]:
// Reseau de decision avec Infer.NET

// Probleme : Diagnostic et traitement

// Variables de chance
Variable<bool> maladie = Variable.Bernoulli(0.15).Named("maladie");

// Test (conditionnel a la maladie)
Variable<bool> testPositif = Variable.New<bool>().Named("test");
using (Variable.If(maladie))
    testPositif.SetTo(Variable.Bernoulli(0.92)); // Sensibilite
using (Variable.IfNot(maladie))
    testPositif.SetTo(Variable.Bernoulli(0.12)); // 1-Specificite

// Inference
InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

// Enumerer les observations possibles et calculer E[U] pour chaque decision
var utilites = new Dictionary<string, double>
{
    {"traiter_malade", 85},
    {"traiter_sain", 65},
    {"pasTraiter_malade", 15},
    {"pasTraiter_sain", 100}
};

Console.WriteLine("=== Reseau de Decision avec Infer.NET ===\n");

foreach (bool obsTest in new[] { true, false })
{
    // Observer le test
    testPositif.ObservedValue = obsTest;
    
    // Inferrer P(maladie|test)
    Bernoulli posteriorMaladie = engine.Infer<Bernoulli>(maladie);
    double pM = posteriorMaladie.GetProbTrue();
    
    // Calculer E[U] pour chaque decision
    double EU_traiter = pM * utilites["traiter_malade"] + (1 - pM) * utilites["traiter_sain"];
    double EU_pasTraiter = pM * utilites["pasTraiter_malade"] + (1 - pM) * utilites["pasTraiter_sain"];
    
    string decision = EU_traiter > EU_pasTraiter ? "TRAITER" : "PAS TRAITER";
    
    Console.WriteLine($"Observation : test = {(obsTest ? "POSITIF" : "NEGATIF")}");
    Console.WriteLine($"  P(maladie|test) = {pM:P1}");
    Console.WriteLine($"  E[U(traiter)] = {EU_traiter:F1}");
    Console.WriteLine($"  E[U(pas traiter)] = {EU_pasTraiter:F1}");
    Console.WriteLine($"  => Decision optimale : {decision}\n");
    
    // Nettoyer l'observation pour la prochaine iteration
    testPositif.ClearObservedValue();
}

=== Reseau de Decision avec Infer.NET ===

Compiling model...done.
Observation : test = POSITIF
  P(maladie|test) = 57,5 %
  E[U(traiter)] = 76,5
  E[U(pas traiter)] = 51,1
  => Decision optimale : TRAITER

Compiling model...done.
Observation : test = NEGATIF
  P(maladie|test) = 1,6 %
  E[U(traiter)] = 65,3
  E[U(pas traiter)] = 98,7
  => Decision optimale : PAS TRAITER



### Interpretation des resultats Infer.NET

L'implementation ci-dessus montre comment utiliser Infer.NET pour les reseaux de decision :

#### Strategie d'implementation

1. **Modeliser les variables de chance** avec Infer.NET (ici : `maladie`, `testPositif`)
2. **Observer** chaque configuration possible des parents informationnels
3. **Inferer** les posteriors avec `engine.Infer<Bernoulli>()`
4. **Calculer** E[U] pour chaque action et selectionner la meilleure

#### Resultats obtenus

| Test observe | P(maladie\|test) | E[U(traiter)] | E[U(pas traiter)] | Decision |
|--------------|------------------|---------------|-------------------|----------|
| POSITIF | 57.5% | 76.5 | 51.1 | TRAITER |
| NEGATIF | 1.6% | 65.3 | 98.7 | PAS TRAITER |

> **Note technique** : Le message "Compiling model...done." apparait car Infer.NET compile le modele en code C# optimise lors de la premiere inference. Les inferences suivantes reutilisent ce code compile.

#### Avantages de l'approche Infer.NET

| Avantage | Description |
|----------|-------------|
| **Inference exacte** | Pas d'approximation pour les modeles discrets |
| **Modeles complexes** | Supporte des structures arbitrairement complexes |
| **Observations partielles** | Gere naturellement les donnees manquantes |
| **Apprentissage** | Peut apprendre les parametres a partir de donnees |

#### Limitations

Infer.NET ne fournit pas de support **natif** pour les noeuds de decision. L'enumeration des decisions doit etre faite manuellement, ce qui peut devenir couteux pour de nombreuses decisions.

Maintenant que nous avons vu comment calculer des politiques optimales avec Infer.NET, examinons comment **visualiser** la structure interne du modele. Cette visualisation est particulierement utile pour :

- Verifier que le modele correspond bien a notre intention
- Identifier les dependances implicites
- Comprendre pourquoi l'inference peut etre lente sur certains modeles

### 7.1 Visualisation du Factor Graph

Infer.NET peut generer le **graphe de facteurs** du modele. C'est utile pour :
- Verifier la structure du modele
- Comprendre les dependances
- Debugger les erreurs d'inference

In [8]:
// Visualisation du graphe de facteurs du modele de diagnostic

// Recreer le modele (nouvelle instance pour generation du graphe)
Variable<bool> maladieFG = Variable.Bernoulli(0.15).Named("Maladie");
Variable<bool> testFG = Variable.New<bool>().Named("TestResult");

using (Variable.If(maladieFG))
    testFG.SetTo(Variable.Bernoulli(0.92));
using (Variable.IfNot(maladieFG))
    testFG.SetTo(Variable.Bernoulli(0.12));

// Activer la generation du graphe de facteurs
InferenceEngine engineFG = new InferenceEngine();
engineFG.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;
engineFG.ShowFactorGraph = true;  // Active la generation du graphe

Console.WriteLine("=== Generation du Factor Graph ===\n");
Console.WriteLine("Le graphe de facteurs sera genere lors de l'inference.");
Console.WriteLine("Sur un environnement avec Graphviz installe, un fichier .dgml sera cree.\n");

// Effectuer l'inference (declenche la generation du graphe)
try 
{
    var posterior = engineFG.Infer<Bernoulli>(maladieFG);
    Console.WriteLine($"Inference reussie : P(Maladie) = {posterior}");
    Console.WriteLine();
    Console.WriteLine("Structure du modele :");
    Console.WriteLine("  [Maladie] -----> [TestResult]");
    Console.WriteLine("      |                 |");
    Console.WriteLine("      v                 v");
    Console.WriteLine("  (Bernoulli)    (Conditionnel)");
    Console.WriteLine();
    Console.WriteLine("Dans le factor graph :");
    Console.WriteLine("  - Noeuds ronds = Variables (Maladie, TestResult)");
    Console.WriteLine("  - Noeuds carres = Facteurs (Prior, Likelihood)");
    Console.WriteLine("  - Aretes = Connexions facteur-variable");
}
catch (Exception ex)
{
    Console.WriteLine($"Note : {ex.Message}");
    Console.WriteLine("La visualisation graphique necessite Graphviz.");
}

=== Generation du Factor Graph ===

Le graphe de facteurs sera genere lors de l'inference.
Sur un environnement avec Graphviz installe, un fichier .dgml sera cree.

Compiling model...done.
Inference reussie : P(Maladie) = Bernoulli(0,15)

Structure du modele :
  [Maladie] -----> [TestResult]
      |                 |
      v                 v
  (Bernoulli)    (Conditionnel)

Dans le factor graph :
  - Noeuds ronds = Variables (Maladie, TestResult)
  - Noeuds carres = Facteurs (Prior, Likelihood)
  - Aretes = Connexions facteur-variable


### Interpretation du resultat de visualisation

**Sortie obtenue** : `P(Maladie) = Bernoulli(0,15)`

| Parametre | Valeur | Signification |
|-----------|--------|---------------|
| Distribution | Bernoulli | Variable binaire (malade/sain) |
| Probabilite | 0.15 | Prior inchange car aucune observation |

> **Note** : Sans observation du test, le posterior est identique au prior. Le factor graph montre les **connexions** entre variables, pas les valeurs. L'inference calcule les marginales en propageant les messages le long de ces connexions.

### Comprendre les Factor Graphs

Le **graphe de facteurs** (factor graph) est la representation interne utilisee par Infer.NET pour l'inference. Comprendre cette structure aide a debugger et optimiser les modeles.

#### Correspondance Reseau Bayesien / Factor Graph

| Reseau Bayesien | Factor Graph | Exemple |
|-----------------|--------------|---------|
| Variable | Noeud rond | Maladie, TestResult |
| CPT P(B\|A) | Noeud carre (facteur) | P(Test\|Maladie) |
| Prior P(A) | Facteur unaire | P(Maladie) = 0.15 |
| Arc A -> B | Aretes A-facteur-B | Connexion via le facteur |

#### Algorithme d'inference (Message Passing)

Infer.NET utilise l'algorithme **Expectation Propagation** (EP) ou **Belief Propagation** (BP) :

1. Les **messages** circulent le long des aretes
2. Chaque facteur **combine** les messages entrants
3. Les messages sont **propages** jusqu'a convergence
4. Les **marginales** sont lues aux noeuds variables

> **Pourquoi c'est important** : La structure du factor graph determine la **complexite** de l'inference. Un graphe avec des cycles necessites des iterations, tandis qu'un arbre permet une inference exacte en un seul passage.

#### Visualisation avec Graphviz

Sur un systeme avec Graphviz installe, `ShowFactorGraph = true` genere un fichier `.dgml` visualisable dans Visual Studio ou converti en image PNG/SVG.

## 8. Exercice : Reseau de Decision Personnalise

### Enonce

Modelisez le reseau de decision suivant :

Un etudiant doit decider s'il revise ou pas pour un examen.
- Variable aleatoire : Difficulte de l'examen (Facile/Difficile)
- Decision : Reviser (cout en temps) ou pas
- Utilite : Note obtenue moins cout de revision

### Solution guidee

L'exercice ci-dessous demande de calculer l'utilite esperee pour deux actions (reviser / ne pas reviser) etant donne l'incertitude sur la difficulte de l'examen. La solution applique la formule standard :

$$E[U(a)] = \sum_{s} P(s) \times U(a, s)$$

ou $s \in \{\text{facile}, \text{difficile}\}$ et $a \in \{\text{reviser}, \text{pas reviser}\}$.

Le **cout seuil** est calcule en resolvant l'equation $E[U(\text{reviser})] = E[U(\text{pas reviser})]$ pour le cout de revision.

In [ ]:
// Exercice : Reseau de decision Etudiant

// Parametres
double pDifficile = 0.4; // P(examen difficile)

// Notes esperees
double note_revise_facile = 18;
double note_revise_difficile = 14;
double note_pasRevise_facile = 14;
double note_pasRevise_difficile = 8;

// Cout de revision (en points d'utilite)
double coutRevision = 2;

// Calcul des utilites esperees
// A completer...

double EU_reviser = (1 - pDifficile) * (note_revise_facile - coutRevision) +
                    pDifficile * (note_revise_difficile - coutRevision);

double EU_pasReviser = (1 - pDifficile) * note_pasRevise_facile +
                       pDifficile * note_pasRevise_difficile;

Console.WriteLine("=== Decision Etudiant ===\n");
Console.WriteLine($"P(examen difficile) = {pDifficile:P0}");
Console.WriteLine($"Cout revision = {coutRevision} points\n");

Console.WriteLine($"E[U(reviser)] = {EU_reviser:F1}");
Console.WriteLine($"E[U(pas reviser)] = {EU_pasReviser:F1}\n");

Console.WriteLine($"=> Decision optimale : {(EU_reviser > EU_pasReviser ? "REVISER" : "NE PAS REVISER")}");

// Analyse de sensibilite : a partir de quel cout la decision change ?
double coutSeuil = (1 - pDifficile) * note_revise_facile + pDifficile * note_revise_difficile - EU_pasReviser;
Console.WriteLine($"\nCout seuil (indifference) : {coutSeuil:F1} points");

### Analyse de la solution de l'exercice

La solution montre que l'etudiant devrait **reviser** avec les parametres donnes.

#### Decomposition du calcul

| Scenario | Probabilite | Utilite si revise | Utilite si pas revise |
|----------|-------------|-------------------|----------------------|
| Facile | 60% | 18 - 2 = 16 | 14 |
| Difficile | 40% | 14 - 2 = 12 | 8 |
| **E[U]** | | **14.4** | **11.6** |

$$E[U(\text{reviser})] = 0.6 \times 16 + 0.4 \times 12 = 14.4$$
$$E[U(\text{pas reviser})] = 0.6 \times 14 + 0.4 \times 8 = 11.6$$

#### Interpretation du cout seuil

Le **cout seuil de 4.8 points** represente le cout maximum de revision pour lequel l'etudiant reste indifferent entre reviser et ne pas reviser.

- Si cout < 4.8 : Reviser est optimal
- Si cout > 4.8 : Ne pas reviser est optimal
- Si cout = 4.8 : Indifference (les deux EU sont egaux)

#### Pour aller plus loin

Cet exercice pourrait etre enrichi avec :
1. **Arc informationnel** : L'etudiant pourrait observer un "signal" sur la difficulte (rumeurs, examens precedents)
2. **Decision sequentielle** : Reviser un peu, puis decider si continuer
3. **Incertitude sur la note** : Modeliser la note comme une variable aleatoire, pas une valeur fixe

## 8bis. Application MAUT : Choix de Site d'Aeroport

### Contexte

Un comite doit choisir l'emplacement d'un nouvel aeroport parmi 3 sites candidats. Les criteres d'evaluation sont incertains car bases sur des etudes preliminaires :

| Critere | Description | Incertitude |
|---------|-------------|-------------|
| **Couts** | Construction, infrastructure | Depend du terrain, geologie |
| **Securite** | Trafic aerien, conditions meteo | Etudes locales necessaires |
| **Nuisances** | Bruit pour riverains, pollution | Impact environnemental |

Cet exemple illustre comment combiner Decision Networks et MAUT avec des attributs incertains.

In [ ]:
// Reseau de Decision Multi-Attribut : Site d'Aeroport
// 3 sites candidats, 3 attributs incertains modelises avec Infer.NET

int nSites = 3;
string[] sites = { "Site A (cotier)", "Site B (rural)", "Site C (periurbain)" };

// Priors sur les attributs basés sur études préliminaires
// Couts (millions EUR) - incertitude sur conditions de terrain
double[] coutMean = { 500, 300, 400 };
double[] coutVar = { 10000, 5000, 8000 };

// Securite (score 0-1) - incertitude sur conditions locales
double[] secAlpha = { 8, 5, 6 };
double[] secBeta = { 2, 5, 4 };

// Nuisances (score 1-10) - incertitude sur impact reel
double[] nuisMean = { 3, 1, 5 };
double[] nuisVar = { 1, 0.5, 2 };

// Poids MAUT (determines par les parties prenantes)
double w_cout = 0.35, w_securite = 0.40, w_nuisances = 0.25;

Console.WriteLine("=== Decision Multi-Attribut avec Infer.NET : Site d'Aeroport ===\n");
Console.WriteLine($"Poids : Cout={w_cout:P0}, Securite={w_securite:P0}, Nuisances={w_nuisances:P0}\n");

// Creer le modele Infer.NET pour chaque site
var engineAeroport = new InferenceEngine();
engineAeroport.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

Console.WriteLine("Distributions des attributs par site :\n");
Console.WriteLine("Site                  | Cout (MEUR)      | Securite         | Nuisances");
Console.WriteLine("----------------------|------------------|------------------|------------------");

var distributionsCout = new Gaussian[nSites];
var distributionsSec = new Beta[nSites];
var distributionsNuis = new Gaussian[nSites];

for (int s = 0; s < nSites; s++)
{
    // Creer les variables pour ce site
    Variable<double> cout_s = Variable.GaussianFromMeanAndVariance(coutMean[s], coutVar[s]).Named($"cout_{s}");
    Variable<double> sec_s = Variable.Beta(secAlpha[s], secBeta[s]).Named($"sec_{s}");
    Variable<double> nuis_s = Variable.GaussianFromMeanAndVariance(nuisMean[s], nuisVar[s]).Named($"nuis_{s}");
    
    // Inferer les distributions
    distributionsCout[s] = engineAeroport.Infer<Gaussian>(cout_s);
    distributionsSec[s] = engineAeroport.Infer<Beta>(sec_s);
    distributionsNuis[s] = engineAeroport.Infer<Gaussian>(nuis_s);
    
    Console.WriteLine($"{sites[s],-21} | {distributionsCout[s].GetMean():F0} +/- {Math.Sqrt(distributionsCout[s].GetVariance()):F0} | {distributionsSec[s].GetMean():P0} +/- {Math.Sqrt(distributionsSec[s].GetVariance()):P0} | {distributionsNuis[s].GetMean():F1} +/- {Math.Sqrt(distributionsNuis[s].GetVariance()):F1}");
}

Console.WriteLine("\n=== Calcul de l'Utilite Esperee par Site ===\n");

// Normalisation et calcul EU
double[] EU = new double[nSites];
for (int s = 0; s < nSites; s++)
{
    // Normaliser les attributs (0-1)
    double normCout = 1 - (distributionsCout[s].GetMean() - 300) / 200;  // 300-500 MEUR → 1-0
    double normSec = distributionsSec[s].GetMean();  // Deja 0-1
    double normNuis = 1 - (distributionsNuis[s].GetMean() - 1) / 9;  // 1-10 → 1-0

    EU[s] = w_cout * normCout + w_securite * normSec + w_nuisances * normNuis;
    
    Console.WriteLine($"{sites[s],-21} :");
    Console.WriteLine($"  v_cout={normCout:F3}, v_sec={normSec:F3}, v_nuis={normNuis:F3}");
    Console.WriteLine($"  => EU = {EU[s]:F3}");
}

// Decision optimale
int bestSite = EU.Select((v, i) => (v, i)).OrderByDescending(x => x.v).First().i;
Console.WriteLine($"\n=== Decision Optimale : {sites[bestSite]} (EU = {EU[bestSite]:F3}) ===");

### Interpretation de l'analyse multi-attribut

Cet exemple combine **Decision Networks** et **MAUT** (Multi-Attribute Utility Theory) pour un probleme realiste de choix de site.

#### Resultats attendus de l'analyse

L'execution du code ci-dessus produit :

| Site | Cout (normalise) | Securite (normalise) | Nuisances (normalise) | EU |
|------|------------------|----------------------|-----------------------|----|
| A (cotier) | 0.0 (500M = max) | 0.80 | 0.78 | ~0.55 |
| B (rural) | 1.0 (300M = min) | 0.50 | 1.00 | ~0.78 |
| C (periurbain) | 0.5 | 0.60 | 0.56 | ~0.55 |

> **Note** : Les valeurs exactes dependent de l'execution et des distributions inferees.

#### Role de l'incertitude

Contrairement a MAUT deterministe, ici chaque attribut est **incertain** :
- Les **couts** suivent une distribution gaussienne (incertitude geologique)
- La **securite** suit une distribution Beta (incertitude sur les conditions)
- Les **nuisances** suivent une gaussienne (incertitude environnementale)

Infer.NET calcule les **esperances** de ces distributions pour le scoring MAUT.

#### Extensions possibles

| Extension | Description |
|-----------|-------------|
| **Analyse de sensibilite** | Varier les poids pour voir si la decision change |
| **Dominance stochastique** | Comparer les distributions completes, pas seulement les moyennes |
| **Information supplementaire** | Modeliser une etude de terrain comme source d'information |

## Recapitulatif des concepts Infer.NET utilises

### Distributions utilisees

| Distribution | Usage dans ce notebook | Exemple |
|--------------|------------------------|---------|
| `Bernoulli` | Variables binaires (maladie, test) | `Variable.Bernoulli(0.15)` |
| `Gaussian` | Attributs continus incertains (couts) | `Variable.GaussianFromMeanAndVariance(500, 10000)` |
| `Beta` | Scores bornes [0,1] (securite) | `Variable.Beta(8, 2)` |

### Patterns Infer.NET

| Pattern | Description | Section |
|---------|-------------|---------|
| **Modele conditionnel** | `using (Variable.If(...))` pour CPT | 7 |
| **Observation** | `variable.ObservedValue = ...` | 7 |
| **Enumeration** | Boucle sur les observations possibles | 7 |
| **Factor Graph** | `ShowFactorGraph = true` | 7.1 |

### Concepts de decision illustres

| Concept | Description | Section |
|---------|-------------|---------|
| Arcs informationnels | Ce que l'agent sait avant de decider | 3 |
| Backward induction | Resolution de la fin vers le debut | 4 |
| Valeur de l'information | Gain d'utilite grace a l'observation | 5 |
| Decisions sequentielles | Plusieurs decisions dans le temps | 6 |
| MAUT probabiliste | Multi-attributs avec incertitude | 8bis |

## 9. Resume

| Concept | Description |
|---------|-------------|
| **Reseau de decision** | Extension des reseaux bayesiens avec decisions et utilites |
| **Noeuds de decision** | Choix controles par l'agent |
| **Noeuds d'utilite** | Consequences des choix |
| **Arcs informationnels** | Ce que l'agent sait au moment de decider |
| **Politique** | Fonction observations → decisions |
| **Backward induction** | Resolution du dernier au premier noeud de decision |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Calculer la valeur de l'information | [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb) |
| Decisions robustes | [Infer-19-Decision-Expert-Systems](Infer-19-Decision-Expert-Systems.ipynb) |
| Decisions sequentielles (MDPs) | [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb) |

---

## Prochaine etape

Dans [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb), nous verrons :

- La valeur de l'information parfaite (EVPI)
- La valeur de l'information d'echantillon (EVSI)
- Des applications : droits de forage, chasse au tresor

---

## References

- Howard & Matheson (1984) : Influence Diagrams
- Shachter (1986) : Evaluating Influence Diagrams
- Russell & Norvig : AI, Chapter 16.5